In [1]:
import yaml
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as f:
        inventory = yaml.safe_load(f)
    return inventory

In [3]:
def configure_loopback(interfaces):
    for interface in interfaces:
        interface_configuration = list()
        interface_configuration.append(f"interface {interface['name']}")
        interface_configuration.append(f"ip address {interface['address']} {interface['netmask']}")
        if interface['enabled'] is True:
            interface_configuration.append('no shutdown')
        else:
            interface_configuration.append('shutdown')
    return interface_configuration

### Main Execution

First thing is read in the inventory file.

In [4]:
if __name__ == '__main__':
    username=input('Enter your username: ')
    password=getpass('Enter your password: ')

Enter your username: wwt
Enter your password: ········


In [5]:
    inventory_file = 'inventory/routers.yml'
    inventory = read_inventory(inventory_file)

### Log into the routers and execute the commands

In [ ]:
    for key, value in inventory.items():
        interface_configuration = configure_loopback(value['interfaces'])
        with Netmiko(host=value['host'], 
                 port=value['port'],
                 username=username, 
                 password=password, 
                 device_type=value['device_type']) as session:
            print(session.find_prompt())
            output = session.send_config_set(interface_configuration)
            print(output)
        